In [1]:
import pandas as pd
import matplotlib.pyplot as plt

import os
import math
import random
import numpy as np
import pickle

In [ ]:
# read all training data
df = pd.read_csv("data/training_set_VU_DM.csv")

In [2]:
# read small training data set
# 49529 rows x 54 columns
df_small = pickle.load(open("pickles/df_small_train.pkl", "rb"))

"Hoge" pearson correlations:
- comp_5_rate_diff vs comp_3_rate_diff
- comp_5_rate_diff vs comp_8_rate_diff
- site_id vs orig_destination_distance
- site_id vs visitor_hist_starrating
- srch_adults_count vs srch_saturday_night_bool
- srch_adults_count vs srch_room_count
- BELANGRIJKSTE: click_bool & booking_bool

In [ ]:
# Kijken naar verband tussen aantal hotels dat weergeven wordt per search en het aantal eisen dat iemand heeft ingevuld
def number_of_hotels_per_search(df):
    """
    Print hoeveel hotels er per search worden weergegeven, print het minimum, maximum, en de unieke waardes
    Handig linkje: https://towardsdatascience.com/data-visualization-exploration-using-pandas-only-beginner-a0a52eb723d5
    """
    
    grouped_searches = df.groupby('srch_id')[['prop_id']].count()
    grouped_searches.sort_values('prop_id').reset_index()
    grouped_searches.columns = ['nr_hotels']
    nr_hotels = grouped_searches['nr_hotels'].unique()
    print(min(nr_hotels))
    print(max(nr_hotels))
    print(sorted(nr_hotels))
    grouped_searches['nr_hotels'].plot(kind='kde')
    
number_of_hotels_per_search(df=df_small)

In [3]:
# kolom toevoegen voor het aantal hotels dat weergegeven wordt per search
counts = {}
for index,row in df_small.iterrows():
    if row['srch_id'] in counts:
        counts[row['srch_id']] += 1
    else:
        counts[row['srch_id']] = 1

df_small['nr_hotels_listed'] = np.nan

for index,row in df_small.iterrows():
    df_small.nr_hotels_listed[index] = counts[row['srch_id']]
    
# kolom toevoegen voor het aantal competitors dat een search heeft

# kolom toevoegen voor het aantal zoek eigenschappen dat een search heeft, dat zijn kolommen:
    # srch_destination_id > altijd ingevuld, en zijn uniek
    # srch_room_count > [1 2 5 4 3 6 7]
    # srch_booking_window > aantal dagen in de toekomst dat de boeking is: misschien hoe lager dit getal, hoe minder opties?
    # srch_length_of_stay > [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 20, 27]
    # srch_adults_count > [2 1 3 5 4 8 6 9]
    # srch_children_count > [1 0 2 3 4 5 6 7]
# heeft dus geen nut, want alle 6 zoekopties zijn altijd ingevuld, maar: wat maakt een zoekopdracht moeilijk?
# bijvoorbeeld veel kinderen (7?!) of veel volwassenen (9!!) of de duur van een boeking (27). Zijn bij deze zoek-
# opdrachten minder resultaten, boeken deze mensen sneller?


In [26]:
# in df_small maar 1 search (id = 315085) met 7 kinderen, hier zijn 31 hotels gedisplayed
ids = []
for index,row in df_small.iterrows():
    if row['nr_hotels_listed'] <= 5:
        if row['srch_id'] not in ids:
            print(f"""id = {row['srch_id']} & srch_room_count = {row['srch_room_count']} 
                  & srch_booking_window = {row['srch_booking_window']} 
                  & srch_destination_id = {row['srch_destination_id']}
                  & srch_length_of_stay = {row['srch_length_of_stay']}
                  & srch_adults_count = {row['srch_adults_count']}
                  & srch_children_count = {row['srch_children_count']}""")
            ids.append(row['srch_id'])

id = 290788 & srch_room_count = 1 
                  & srch_booking_window = 0 
                  & srch_destination_id = 1306
                  & srch_length_of_stay = 4
                  & srch_adults_count = 2
                  & srch_children_count = 0
id = 6400 & srch_room_count = 1 
                  & srch_booking_window = 321 
                  & srch_destination_id = 16504
                  & srch_length_of_stay = 6
                  & srch_adults_count = 2
                  & srch_children_count = 1
id = 325372 & srch_room_count = 6 
                  & srch_booking_window = 16 
                  & srch_destination_id = 8859
                  & srch_length_of_stay = 2
                  & srch_adults_count = 6
                  & srch_children_count = 0
id = 278925 & srch_room_count = 1 
                  & srch_booking_window = 1 
                  & srch_destination_id = 17443
                  & srch_length_of_stay = 1
                  & srch_adults_count = 1
             

id = 84386 & srch_room_count = 1 
                  & srch_booking_window = 24 
                  & srch_destination_id = 14053
                  & srch_length_of_stay = 1
                  & srch_adults_count = 2
                  & srch_children_count = 0
id = 167697 & srch_room_count = 1 
                  & srch_booking_window = 57 
                  & srch_destination_id = 9073
                  & srch_length_of_stay = 2
                  & srch_adults_count = 2
                  & srch_children_count = 0
id = 103617 & srch_room_count = 2 
                  & srch_booking_window = 3 
                  & srch_destination_id = 22145
                  & srch_length_of_stay = 1
                  & srch_adults_count = 2
                  & srch_children_count = 0
id = 281977 & srch_room_count = 1 
                  & srch_booking_window = 1 
                  & srch_destination_id = 1097
                  & srch_length_of_stay = 3
                  & srch_adults_count = 2
             

In [ ]:
# what can predict 'position', so we can use this in the test data
